In [1]:
import os
import sys
import pandas as pd

TOP_DIR=os.path.abspath("../../")
if not TOP_DIR in sys.path: sys.path.append(TOP_DIR)

In [2]:
from prepare import LMS_EXTRACT, DASHBOARD_DIR, create_table, save_files, summarise, labour_market_status_variables, long_term_unemployed_variables
from scripts.util.metadata import read_meta, extract_dates

Load in the LMS data

In [3]:
lms_extract = pd.read_csv(
    LMS_EXTRACT,
    parse_dates=['lms_period']
)

Extract relevant metrics, pivot and save file for labour market

In [4]:
labour_market_status = lms_extract.pipe(
    create_table, labour_market_status_variables
).pipe(
    save_files, 'labour_market_status'
)
labour_market_status

variable,age_16_to_17_not_in_ft_education_in_employment_sa,age_18_to_24_not_in_ft_education_in_employment_sa,age_16_to_24_not_in_ft_education_in_employment_sa,men_16_to_24_not_in_ft_education_in_employment_sa,women_16_to_24_not_in_ft_education_in_employment_sa,age_16_to_24_in_ft_education_in_employment_sa,age_16_to_17_not_in_ft_education_unemployed_sa,age_18_to_24_not_in_ft_education_unemployed_sa,age_16_to_24_not_in_ft_education_unemployed_sa,men_16_to_24_not_in_ft_education_unemployed_sa,...,men_16_to_24_not_in_ft_education_total_sa,women_16_to_24_not_in_ft_education_total_sa,unemployment_rate_sa,economic_inactivity_rate_sa,age_16_to_24_in_employment_sa,age_16_to_24_unemployed_sa,age_16_to_24_economically_inactive_sa,age_16_to_24_unemployment_rate_sa,quarter_label,quarter_axis_label
lms_period,,,,,,,,,,,,,,,,,,,,,
2000-02-01,251.0,2756.0,3006.0,1638.0,1369.0,925.0,78.0,335.0,413.0,241.0,...,2025.0,1959.0,5.9,23.2,3931.0,581.0,1748.0,12.9,Jan-Mar 2000,Jan-Mar\n2000
2000-05-01,242.0,2723.0,2965.0,1612.0,1353.0,955.0,76.0,330.0,406.0,249.0,...,2016.0,1918.0,5.6,23.3,3921.0,552.0,1792.0,12.3,Apr-Jun 2000,Apr-Jun\n2000
2000-08-01,234.0,2755.0,2989.0,1626.0,1363.0,956.0,85.0,316.0,401.0,231.0,...,2023.0,1933.0,5.4,23.2,3945.0,545.0,1795.0,12.1,Jul-Sep 2000,Jul-Sep\n2000
2000-11-01,233.0,2744.0,2977.0,1625.0,1352.0,927.0,77.0,318.0,395.0,236.0,...,2013.0,1933.0,5.3,23.5,3903.0,548.0,1863.0,12.3,Oct-Dec 2000,Oct-Dec\n2000
2001-02-01,245.0,2744.0,2989.0,1628.0,1361.0,953.0,80.0,322.0,402.0,248.0,...,2035.0,1936.0,5.1,23.4,3942.0,536.0,1866.0,12.0,Jan-Mar 2001,Jan-Mar\n2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-01,81.0,2682.0,2764.0,1473.0,1291.0,1035.0,22.0,341.0,362.0,249.0,...,1989.0,1783.0,4.3,21.4,3798.0,533.0,2741.0,12.3,Apr-Jun 2023,Apr-Jun\n2023
2023-08-01,83.0,2551.0,2635.0,1382.0,1253.0,1007.0,15.0,310.0,325.0,226.0,...,1928.0,1770.0,4.2,21.9,3641.0,499.0,2952.0,12.1,Jul-Sep 2023,Jul-Sep\n2023
2023-11-01,98.0,2651.0,2749.0,1461.0,1288.0,930.0,18.0,296.0,314.0,209.0,...,2015.0,1780.0,3.9,21.9,3678.0,481.0,2956.0,11.6,Oct-Dec 2023,Oct-Dec\n2023


Extract relevant metrics for long-term unemployment, pivot and save file

In [5]:
def construct_16_24_stats(data):
    data['age_16_to_24_unemployed_sa'] = (data.age_18_to_24_unemployed_sa + data.age_16_to_17_unemployed_sa).round(0)
    data['age_16_to_24_unemployed_6_to_12_months_sa'] = (data.age_18_to_24_unemployed_6_to_12_months_sa + data.age_16_to_17_unemployed_6_to_12_months_sa).round(0)
    data['age_16_to_24_unemployed_over_12_months_sa'] = (data.age_18_to_24_unemployed_over_12_months_sa + data.age_16_to_17_unemployed_over_12_months_sa).round(0)
    data['age_16_to_24_unemployed_6_to_12_months_rate_sa'] = (data['age_16_to_24_unemployed_6_to_12_months_sa'] / \
      data['age_16_to_24_unemployed_sa'] * 100).round(1)
    data['age_16_to_24_unemployed_over_12_months_rate_sa'] = (data['age_16_to_24_unemployed_over_12_months_sa'] / \
      data['age_16_to_24_unemployed_sa'] * 100).round(1)

    # Handle missing data by using the 18-24 figure instead
    data['missing_over_12_months_data'] = data.age_16_to_24_unemployed_over_12_months_sa.isna()
    data.age_16_to_24_unemployed_over_12_months_sa = data.age_16_to_24_unemployed_over_12_months_sa.mask(
        data.missing_over_12_months_data,
        data.age_18_to_24_unemployed_over_12_months_sa
    )
    data.age_16_to_24_unemployed_over_12_months_rate_sa = data.age_16_to_24_unemployed_over_12_months_rate_sa.mask(
        data.missing_over_12_months_data,
        data.age_18_to_24_unemployed_over_12_months_rate_sa.round(1)
    )
    return data

def add_axis_labels(data):
    data['quarter_axis_label'] = data.quarter_label.str.replace(' ', '\\n')
    return data    

In [6]:
long_term_unemployed = lms_extract.pipe(create_table, long_term_unemployed_variables).pipe(construct_16_24_stats).pipe(add_axis_labels)
long_term_unemployed.pipe(save_files, 'long_term_unemployed')

variable,age_16_to_17_unemployed_sa,age_18_to_24_unemployed_sa,age_16_to_17_unemployed_6_to_12_months_sa,age_16_to_17_unemployed_over_12_months_sa,age_16_to_17_unemployed_over_12_months_rate_sa,age_18_to_24_unemployed_6_to_12_months_sa,age_18_to_24_unemployed_over_12_months_sa,age_18_to_24_unemployed_over_12_months_rate_sa,quarter_label,quarter_axis_label,age_16_to_24_unemployed_sa,age_16_to_24_unemployed_6_to_12_months_sa,age_16_to_24_unemployed_over_12_months_sa,age_16_to_24_unemployed_6_to_12_months_rate_sa,age_16_to_24_unemployed_over_12_months_rate_sa,missing_over_12_months_data
lms_period,,,,,,,,,,,,,,,,
2000-02-01,175.0,407.0,23.0,8.0,4.8,60.0,57.0,13.9,Jan-Mar 2000,Jan-Mar\n2000,582.0,83.0,65.0,14.3,11.2,False
2000-05-01,165.0,386.0,26.0,10.0,6.2,56.0,64.0,16.5,Apr-Jun 2000,Apr-Jun\n2000,551.0,82.0,74.0,14.9,13.4,False
2000-08-01,173.0,373.0,23.0,13.0,7.3,53.0,61.0,16.4,Jul-Sep 2000,Jul-Sep\n2000,546.0,76.0,74.0,13.9,13.6,False
2000-11-01,161.0,387.0,21.0,11.0,6.6,60.0,50.0,12.8,Oct-Dec 2000,Oct-Dec\n2000,548.0,81.0,61.0,14.8,11.1,False
2001-02-01,153.0,383.0,16.0,10.0,6.5,56.0,55.0,14.4,Jan-Mar 2001,Jan-Mar\n2001,536.0,72.0,65.0,13.4,12.1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-01,93.0,440.0,18.0,3.0,3.8,82.0,59.0,13.3,Apr-Jun 2023,Apr-Jun\n2023,533.0,100.0,62.0,18.8,11.6,False
2023-08-01,75.0,424.0,15.0,NaN,NaN,81.0,52.0,12.3,Jul-Sep 2023,Jul-Sep\n2023,499.0,96.0,52.0,19.2,12.3,True
2023-11-01,104.0,378.0,10.0,NaN,NaN,58.0,64.0,16.9,Oct-Dec 2023,Oct-Dec\n2023,482.0,68.0,64.0,14.1,16.9,True


Create a summary file

In [7]:
summary = pd.concat([
    summarise(
      long_term_unemployed=long_term_unemployed,
      labour_market_status=labour_market_status,
    ),
    read_meta().pipe(extract_dates, 'LMS'),
])

summary.to_json(os.path.join(DASHBOARD_DIR, 'latest.json'), indent=2, date_format='iso')

In [8]:
summary

unemployment_rate_all_working_age                                      4.2
economic_inactivity_rate_all_working_age                              22.2
unemployment_rate_young_people                                        13.4
economic_inactivity_rate_young_people                                 41.7
unemployment_rate_not_in_full_time_education                          11.9
economic_inactivity_rate_not_in_full_time_education                   19.4
long_term_unemployment_rate_6_to_12_months                            12.9
long_term_unemployment_rate_over_12_months                            93.0
most_recent_lms_period                                        Apr-Jun 2024
last_update                                            2024-08-13 00:00:00
next_update                                            2024-10-09 00:00:00
dtype: object